In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import calendar
from pandas_profiling import ProfileReport
#import warnings; 
#warnings.simplefilter('ignore')
import seaborn as sns
import numpy as np

%matplotlib inline
plt.rcParams.update({'font.size': 16, 'figure.figsize': (16,6)})




In [2]:
import random

filename = 'data/training_set_VU_DM.csv'
p = 1  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(
         filename,
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)

#df = pd.read_csv(filename, nrows=1000)
df['date_time'] = pd.to_datetime(df['date_time'])

df_test = pd.read_csv('data/test_set_VU_DM.csv', header=0, skiprows=lambda i: i>0 and random.random() > p)
df_test['date_time'] = pd.to_datetime(df_test['date_time'])

In [4]:
df.columns

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

### Average numeric features over:
srch_id\
visitor_location_country_id\
prop_id\
srch_destination_id\
prop_starrating

In [4]:
# add the average of these features over some column 
# for example: calc avg of these columns for every search_id
numeric_features = ['visitor_hist_starrating', 'visitor_hist_adr_usd',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'click_bool', 'gross_bookings_usd',
       'booking_bool']

In [5]:
# groupy by these columns
avg_over_columns = ['srch_id', 'visitor_location_country_id', 'prop_id', 'prop_starrating', 'srch_destination_id']
df_groupbys = []

# first create aggreagted all dataframes over specified columns, adjust column names
for split_col in avg_over_columns:
    df_groupby = df.groupby(split_col)[numeric_features].mean()

    new_colnames = []
    for i,col in enumerate(df_groupby.columns):
        new_colnames.append('mean_' + col + '_over_' + split_col)
    df_groupby.columns = new_colnames
    df_groupbys.append(df_groupby)
    
# merge oridinal df with the aggregated dataframes
for col, df_groupby in zip(avg_over_columns, df_groupbys):
    df = pd.merge(df, df_groupby, right_index=True, left_on = col, how='left')

In [ ]:
#TODO
#calculate diff between price_usd and gross_bookings_usd